In [20]:
import sys
import requests
import json
import pandas as pd
from datetime import datetime, timedelta

from pprint import PrettyPrinter
pp = PrettyPrinter(indent=4)



In [21]:
def adjust_sensors_delay_time(implementation):
    """
    This function returns the delay time for each sensor in order to adjust the data
    """
    #esp32
    seconds_delay = 0
    milliseconds_delay = 0
    if implementation == "ESP32S3": 
        seconds_delay = 2
        milliseconds_delay = 920

    return timedelta(seconds=seconds_delay, milliseconds=milliseconds_delay)

In [22]:
def jobs_unix_timestamp(implementation, day, month):
    epoch = datetime.utcfromtimestamp(0)

    #startDT = datetime(2023, month, 11, 19, 0, 0, 0) - timedelta(hours=1) + adjust_sensors_delay_time(implementation)
    startDT = datetime(2023, month, day, 9, 0, 0, 0) - timedelta(hours=1) + adjust_sensors_delay_time(implementation)
    startTS = int((startDT - epoch).total_seconds() * 1000.0)
    #endDT = datetime(2023, month, 12, 7, 0, 0 , 0) - timedelta(hours=1) + adjust_sensors_delay_time(implementation)
    endDT = datetime(2023, month, day, 18, 0, 0 , 0) - timedelta(hours=1) + adjust_sensors_delay_time(implementation)
    endTS = int((endDT - epoch).total_seconds() * 1000.0)
    

    return str(startTS), str(endTS)

In [23]:
RFID_analysis_keys = [
    'gabarit'
]

TASK_analysis_keys = [
    'task', 'empid', 'sec'
]
#TASKTOTAL_analysis_keys = [
#    'emp_id', 'taskid','totalm', 'av','pr' 
#]
TASKDISTANCE_analysis_keys= [
    'distance',
]
BLE_analysis_keys = [
    'id','rssi'
]


BLE_analysis_keys = [
    'id','rssi'
]


In [24]:
def json_csv(data, dataset, implementation):
    data_df = pd.DataFrame()
    first_flag = True
    
    if dataset == "rfid":
        analysis_keys = RFID_analysis_keys
    elif dataset == "task":
        analysis_keys = TASK_analysis_keys
    elif dataset == "taskdistance":
        analysis_keys = TASKDISTANCE_analysis_keys
    else:
        analysis_keys = BLE_analysis_keys



    for analysis_key in analysis_keys:
        if first_flag:
            data_df = pd.DataFrame(data[analysis_key])
            data_df = data_df.rename(columns={'value': analysis_key})
            first_flag = False
        else:
            data_df = data_df.join(pd.DataFrame(data[analysis_key]).set_index('ts'), on='ts')
            data_df = data_df.rename(columns={'value': analysis_key})
    # change the timestamp to seconds
    data_df['ts'] = pd.to_datetime(data_df['ts']/1000,unit='s') + timedelta(hours=1) - adjust_sensors_delay_time(implementation)
    data_df['ts']= data_df['ts'].astype('datetime64[s]')
    return data_df.iloc[::-1]

def getToken():
    username = 'isabel.portugal@tecnico.ulisboa.pt' #email used to create your ThingsBoard account (your email associated with tennant)
    password = 'tHn%Mv1pnmq2av='		#password of your ThingsBoard account (or your tennant account)
    url = 'http://iot.istartlab.tecnico.ulisboa.pt/api/auth/login'
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    loginJSON = {'username': username, 'password': password}
    tokenAuthResp = requests.post(url, headers=headers, json=loginJSON).json()
    #print(tokenAuthResp)
    token = tokenAuthResp['token']
    return token

In [25]:
tkn = getToken()

my_headers = {'X-Authorization':  "Bearer " + tkn, "Content-Type" : "application/json"}
#print(my_headers)
#url to test connection
url = "http://iot.istartlab.tecnico.ulisboa.pt/api/plugins/telemetry/DEVICE/27110c40-2a6b-11ee-9b43-67453a5ac630/keys/timeseries"

response = requests.get(url, headers=my_headers)
print(response) , print(response.text)

my_headers = {'X-Authorization':  "Bearer " + tkn, "Content-Type" : "application/json"}
#print(my_headers)

<Response [404]>
Device with requested id wasn't found!


In [56]:

def get_response(day_int,month_int,dataset):

    day_str = str(day_int).zfill(2)
    month_str = str(month_int).zfill(2)
    date = day_str + "_" + month_str
    print(date)

    startTS, endTS = jobs_unix_timestamp("ESP32S3", day_int, month_int)

    limit = "1000000000"

    date_filename = dataset + "_" + day_str + "_" + month_str

    cnt = 1

    if dataset == "rfid":
        devices = ["27110c40-2a6b-11ee-9b43-67453a5ac633","f4765800-2cc5-11ee-a22d-b5d6c11ab119","10fc9700-2cc6-11ee-a22d-b5d6c11ab119"]
        #for analysis_key in RFID_analysis_keys:
        keys = "ts,gabarit"

    elif dataset == "task":
        devices = ["502ba360-d205-11ed-a394-2dc30865e203"]
        #for analysis_key in TASK_analysis_keys:
        keys = "task,empid,sec" 
        
    elif dataset == "taskdistance":
        devices = ["502ba360-d205-11ed-a394-2dc30865e203"]
        keys = "distance"

    else:
        devices = ["8516caf0-7a71-11ed-a394-2dc30865e203","fe7d6ff0-b52e-11ed-a394-2dc30865e203","4b593330-2498-11ee-a394-2dc30865e203" ]
        #1: fim da linha, 2: pacotes, 3: elevator
        #for analysis_key in BLE_analysis_keys:
        keys = "id,rssi"   

    print(keys)

    for dev in devices:
        url = "http://iot.istartlab.tecnico.ulisboa.pt/api/plugins/telemetry/DEVICE/" + dev + "/values/timeseries?keys=" + keys + "&startTs=" + startTS + "&endTs=" + endTS + "&limit=" + limit
        response = requests.get(url, headers=my_headers)
        print(response.text)
        #if response.text:
        #    print("No data found in that day")
        #    return
        if len(response.text) == 2:
            print("No data found in that day")
            return
        print("Response received", response.text)
        data = json.loads(response.text)
        data_df = json_csv(data, dataset, "ESP32S3")
        if dataset == "rfid" or dataset == "ble":
            data_df.to_csv(date_filename + "_" + str(cnt) + ".csv")
        else:
            data_df.to_csv(date_filename + ".csv")
                    #
        cnt += 1

    print("done")
    return

In [59]:
# CHANGE ONLY THIS
#31.07 ate 30.09
dataset = "taskdistance"#"task"

if dataset == "taskdistance":
    start_date = '31/07/2023'
    end_date = '31/07/2023'
    start_date = pd.to_datetime(start_date, format='%d/%m/%Y')
    end_date = pd.to_datetime(end_date, format='%d/%m/%Y')
    date_range = pd.date_range(start=start_date, end=end_date)
    for date in date_range:
        day_int = date.day
        month_int = date.month
        get_response(day_int,month_int,dataset)
        print()
else:
    day_int = 27
    month_int = 9
    get_response(day_int,month_int,dataset)

31_07
distance
{"distance":[{"ts":1690822749011,"value":"251"},{"ts":1690822688151,"value":"248"},{"ts":1690822627368,"value":"250"},{"ts":1690822566582,"value":"248"},{"ts":1690822505788,"value":"248"},{"ts":1690822445000,"value":"252"},{"ts":1690822384304,"value":"277"},{"ts":1690822323411,"value":"270"},{"ts":1690822262620,"value":"271"},{"ts":1690822201948,"value":"273"},{"ts":1690822141135,"value":"261"},{"ts":1690822080238,"value":"263"},{"ts":1690822019446,"value":"239"},{"ts":1690821987025,"value":"426"},{"ts":1690821954602,"value":"578"},{"ts":1690821939407,"value":"419"},{"ts":1690821922185,"value":"573"},{"ts":1690821902936,"value":"395"},{"ts":1690821868497,"value":"603"},{"ts":1690821853295,"value":"279"},{"ts":1690821838096,"value":"526"},{"ts":1690821822902,"value":"326"},{"ts":1690821807702,"value":"525"},{"ts":1690821792506,"value":"303"},{"ts":1690821762116,"value":"611"},{"ts":1690821746919,"value":"416"},{"ts":1690821720577,"value":"647"},{"ts":1690821697283,"value"

In [27]:
print(response.text)

{"distance":[{"ts":1690995589709,"value":"740"},{"ts":1690995528861,"value":"737"},{"ts":1690995468090,"value":"740"},{"ts":1690995407508,"value":"765"},{"ts":1690995392130,"value":"491"},{"ts":1690995376014,"value":"730"},{"ts":1690995354933,"value":"403"},{"ts":1690995317191,"value":"626"},{"ts":1690995301991,"value":"385"},{"ts":1690995277824,"value":"620"},{"ts":1690995262492,"value":"274"},{"ts":1690995217927,"value":"583"},{"ts":1690995202805,"value":"374"},{"ts":1690995158172,"value":"548"},{"ts":1690995142982,"value":"148"},{"ts":1690995110649,"value":"493"},{"ts":1690995095379,"value":"671"},{"ts":1690995079180,"value":"449"},{"ts":1690995061036,"value":"644"},{"ts":1690995043737,"value":"464"},{"ts":1690994982969,"value":"502"},{"ts":1690994963806,"value":"686"},{"ts":1690994948535,"value":"472"},{"ts":1690994921203,"value":"631"},{"ts":1690994905069,"value":"472"},{"ts":1690994889798,"value":"1261"},{"ts":1690994874608,"value":"492"},{"ts":1690994855571,"value":"677"},{"ts":